# This whole notebook will be data analysis, which most will be contributed to my written piece

## import the female list I curated

In [187]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd

In [8]:
df_w = pd.read_csv('females_list_bbc.csv')

In [143]:
df_w

,title,director,year,link,rating,wins,nominations,genres,Number_of_Genres,Era,Broad_Genre
0,The Kids are All Right,Lisa Cholodenko,2010,https://www.imdb.com/title/tt0842926/,7.0,29.0,133.0,"['Comedy', 'Drama', 'Romance']",3,2011-2015,Comedy
1,The Souvenir,Joanna Hogg,2019,https://www.imdb.com/title/tt6920356/,6.4,7.0,37.0,"['Coming-of-Age', 'Period Drama', 'Drama', 'Ro...",4,2016-2020,Drama
2,Somewhere,Sofia Coppola,2010,https://www.imdb.com/title/tt1421051/,6.3,4.0,8.0,"['Dark Comedy', 'Quirky Comedy', 'Comedy', 'Dr...",5,2011-2015,Comedy
3,Adoption,Márta Mészáros,1975,https://www.imdb.com/title/tt0073948/,7.2,4.0,1.0,['Drama'],1,before 2000,Drama
4,The Meetings of Anna,Chantal Akerman,1977,https://www.imdb.com/title/tt4441280/,5.9,NaN,NaN,"['Drama', 'Thriller']",2,before 2000,Drama
5,Ritual in Transfigured Time,Maya Deren,1946,https://www.imdb.com/title/tt0038885/,6.9,NaN,NaN,"['Fantasy', 'Short']",2,before 2000,Other
6,News From Home,Chantal Akerman,1977,https://www.imdb.com/title/tt0076452/,7.3,NaN,NaN,['Documentary'],1,before 2000,Documentary
7,Red Road,Andrea Arnold,2006,https://www.imdb.com/title/tt0471030/,6.8,22.0,12.0,"['Drama', 'Mystery', 'Thriller']",3,2006-2010,Drama
8,Raw,Julia Ducournau,2016,https://www.imdb.com/title/tt4954522/,6.9,25.0,45.0,"['Body Horror', 'Coming-of-Age', 'Drama', 'Hor...",4,2016-2020,Drama
9,White Material,Claire Denis,2009,https://www.imdb.com/title/tt1135952/,6.9,1.0,10.0,"['Drama', 'War']",2,2006-2010,Drama


### convert ratings, nominations, wins to numerics

In [10]:
df_w['rating'] = pd.to_numeric(df_w['rating'], errors='coerce')

In [11]:
df_w['wins'] = pd.to_numeric(df_w['wins'], errors='coerce')

In [12]:
df_w['nominations'] = pd.to_numeric(df_w['nominations'], errors='coerce')

### the mean rating for films directed by females

In [13]:
df_w['rating'].median()

7.3

### the number of awards won on average

In [14]:
df_w['wins'].median()

12.0

### the number of nominations won on average

In [15]:
df_w['nominations'].median()

15.0

In [16]:
new_df = pd.DataFrame({
    'females': ['females'] * len(df_w),  # Repeat "females" for each row
    'rating': df_w['rating'],
    'wins': df_w['wins'],
    'director':df_w['director']
})

In [40]:
df_w['Number_of_Genres'] = df_w['genres'].str.strip("[]").str.split(", ").apply(len)


In [45]:
bins = [0, 1999, 2005, 2010, 2015, 2020]  # 0 represents "before 2000"
labels = ['before 2000', '2000-2005', '2006-2010', '2011-2015', '2016-2020']

# Add 'Era' column
df_w['Era'] = pd.cut(
    df_w['year'],
    bins=bins,
    labels=labels,
    right=False
)

In [138]:
df_w

,title,director,year,link,rating,wins,nominations,genres,Number_of_Genres,Era,Broad_Genre
0,The Kids are All Right,Lisa Cholodenko,2010,https://www.imdb.com/title/tt0842926/,7.0,29.0,133.0,"['Comedy', 'Drama', 'Romance']",3,2011-2015,Comedy
1,The Souvenir,Joanna Hogg,2019,https://www.imdb.com/title/tt6920356/,6.4,7.0,37.0,"['Coming-of-Age', 'Period Drama', 'Drama', 'Ro...",4,2016-2020,Drama
2,Somewhere,Sofia Coppola,2010,https://www.imdb.com/title/tt1421051/,6.3,4.0,8.0,"['Dark Comedy', 'Quirky Comedy', 'Comedy', 'Dr...",5,2011-2015,Comedy
3,Adoption,Márta Mészáros,1975,https://www.imdb.com/title/tt0073948/,7.2,4.0,1.0,['Drama'],1,before 2000,Drama
4,The Meetings of Anna,Chantal Akerman,1977,https://www.imdb.com/title/tt4441280/,5.9,NaN,NaN,"['Drama', 'Thriller']",2,before 2000,Drama
5,Ritual in Transfigured Time,Maya Deren,1946,https://www.imdb.com/title/tt0038885/,6.9,NaN,NaN,"['Fantasy', 'Short']",2,before 2000,Other
6,News From Home,Chantal Akerman,1977,https://www.imdb.com/title/tt0076452/,7.3,NaN,NaN,['Documentary'],1,before 2000,Documentary
7,Red Road,Andrea Arnold,2006,https://www.imdb.com/title/tt0471030/,6.8,22.0,12.0,"['Drama', 'Mystery', 'Thriller']",3,2006-2010,Drama
8,Raw,Julia Ducournau,2016,https://www.imdb.com/title/tt4954522/,6.9,25.0,45.0,"['Body Horror', 'Coming-of-Age', 'Drama', 'Hor...",4,2016-2020,Drama
9,White Material,Claire Denis,2009,https://www.imdb.com/title/tt1135952/,6.9,1.0,10.0,"['Drama', 'War']",2,2006-2010,Drama


In [151]:
df_w.sort_values(['Broad_Genre', 'wins'], ascending=[True, True])  


,title,director,year,link,rating,wins,nominations,genres,Number_of_Genres,Era,Broad_Genre
10,Fast Times at Ridgemont High,Amy Heckerling,1982,https://www.imdb.com/title/tt0083929/,7.1,1.0,1.0,"['Teen Comedy', 'Comedy', 'Drama']",3,before 2000,Comedy
54,Near Dark,Kathryn Bigelow,1987,https://www.imdb.com/title/tt0093605/,6.9,1.0,9.0,"['Contemporary Western', 'Dark Comedy', 'Vampi...",4,before 2000,Comedy
94,Daisies,Věra Chytilová,1966,https://www.imdb.com/title/tt0060959/,7.2,1.0,1.0,"['Coming-of-Age', 'Dark Comedy', 'Satire', 'Sc...",7,before 2000,Comedy
42,Desperately Seeking Susan,Susan Seidelman,1985,https://www.imdb.com/title/tt0089017/,6.1,2.0,6.0,"['Quirky Comedy', 'Screwball Comedy', 'Comedy'...",4,before 2000,Comedy
83,Seven Beauties,Lina Wertmüller,1975,https://www.imdb.com/title/tt0075040/,7.7,2.0,9.0,"['Comedy', 'Drama', 'War']",3,before 2000,Comedy
98,Cléo from 5 to 7,Agnès Varda,1962,https://www.imdb.com/title/tt0055852/,7.8,2.0,1.0,"['Psychological Drama', 'Comedy', 'Drama', 'Mu...",4,before 2000,Comedy
2,Somewhere,Sofia Coppola,2010,https://www.imdb.com/title/tt1421051/,6.3,4.0,8.0,"['Dark Comedy', 'Quirky Comedy', 'Comedy', 'Dr...",5,2011-2015,Comedy
17,Sleepless in Seattle,Nora Ephron,1993,https://www.imdb.com/title/tt0108160/,6.8,4.0,19.0,"['Feel-Good Romance', 'Romantic Comedy', 'Roma...",6,before 2000,Comedy
40,A League of their Own,Penny Marshall,1992,https://www.imdb.com/title/tt0104694/,7.4,6.0,13.0,"['Baseball', 'Comedy', 'Drama', 'Sport']",4,before 2000,Comedy
80,Clueless,Amy Heckerling,1995,https://www.imdb.com/title/tt0112697/,6.9,6.0,12.0,"['Coming-of-Age', 'Feel-Good Romance', 'Romant...",8,before 2000,Comedy


In [49]:
df_w['genres'] = df_w['genres'].str.strip("[]").str.replace("'", "").str.split(", ")


In [50]:
df_exploded = df_w.explode('genres')

In [53]:
genre_counts = df_exploded.groupby(['Era', 'genres']).size().reset_index(name='Count')


In [59]:
genre_counts = genre_counts[genre_counts['Count'] > 0]


In [68]:
genre_counts

,Era,genres,Count
0,before 2000,Action,1
2,before 2000,Adventure,1
3,before 2000,Adventure Epic,1
5,before 2000,Baseball,1
6,before 2000,Biography,3
...,...,...,...
393,2016-2020,Steamy Romance,1
397,2016-2020,Suspense Mystery,1
399,2016-2020,Teen Comedy,1
402,2016-2020,Thriller,1


In [74]:
genre_counts['genres'] = genre_counts['genres'].astype(str)  # Force string conversion


In [156]:
parent_genres = {
    'Documentary': ['documentary', 'docu', 'biography'],
    'Comedy': ['comedy', 'humor', 'satire', 'dark comedy'],
    'Psychological Drama' : ['psychological drama'],
    'Horror': ['horror', 'terror', 'monster', 'slasher', 'ghost'],
    'Sports': ['sport', 'ball', 'baseball', 'athlete', 'fitness'],
    'Romance': ['romance', 'romantic', 'love', 'steamy'],
    'Thriller': ['thriller', 'suspense', 'crime'],
    'Coming-of-Age': ['coming-of-age', 'teen', 'youth', 'adolescent'],
    'Mystery': ['mystery', 'whodunit', 'enigma'],
    'Crime': ['crime', 'heist', 'gangster', 'mafia', 'noir'],
    'Sci-Fi': ['sci-fi', 'science fiction', 'space opera', 'cyberpunk', 'artificial', 'fantasy'],
    'Family': ['family'],
    'History': ['history'],
    'War' : ['war'],
    'Action': ['action', 'adventure', 'epic', 'heroic'],
    'Animation': ['anim'],
    'Other': []
}

def map_to_parent(genre):
    if pd.isna(genre):
        return 'Other'
    genre_lower = str(genre).lower().strip()
    for parent, keywords in parent_genres.items():
        if any(keyword in genre_lower for keyword in keywords):  # ✅ Correct
            return parent
    return 'Other'

# Apply mapping
genre_counts['Broad_Genre'] = genre_counts['genres'].apply(map_to_parent)

# Aggregate
df_w_agg = genre_counts.groupby(['Era', 'Broad_Genre'], as_index=False)['Count'].sum()

In [157]:
genre_counts

,Era,genres,Count,Broad_Genre
0,before 2000,Action,1,Action
2,before 2000,Adventure,1,Action
3,before 2000,Adventure Epic,1,Action
5,before 2000,Baseball,1,Sports
6,before 2000,Biography,3,Documentary
8,before 2000,Body Swap Comedy,1,Comedy
9,before 2000,Comedy,9,Comedy
10,before 2000,Coming-of-Age,5,Coming-of-Age
11,before 2000,Contemporary Western,1,Other
12,before 2000,Costume Drama,1,Other


In [158]:
df_sorted = df_w_agg.sort_values(['Era', 'Count'], ascending=[True, False])

In [159]:
df_sorted

,Era,Broad_Genre,Count
10,before 2000,Other,54
2,before 2000,Comedy,23
5,before 2000,Documentary,16
12,before 2000,Romance,15
15,before 2000,Thriller,11
13,before 2000,Sci-Fi,9
0,before 2000,Action,7
11,before 2000,Psychological Drama,7
3,before 2000,Coming-of-Age,5
4,before 2000,Crime,4


In [181]:
df_sorted.to_csv('df_sorted.csv', index=False)


In [163]:
df_sorted['Gender'] = 'Female' 


In [164]:
df_sorted

,Era,Broad_Genre,Count,Gender
10,before 2000,Other,54,Female
2,before 2000,Comedy,23,Female
5,before 2000,Documentary,16,Female
12,before 2000,Romance,15,Female
15,before 2000,Thriller,11,Female
13,before 2000,Sci-Fi,9,Female
0,before 2000,Action,7,Female
11,before 2000,Psychological Drama,7,Female
3,before 2000,Coming-of-Age,5,Female
4,before 2000,Crime,4,Female


## import the list of 100 greatest film (regardless of gender)

In [19]:
df_all = pd.read_csv('list_all_bbc.csv')

In [20]:
df_all

,title,director,year,link,rating,wins,nominations,genres
0,Toni Erdmann,Maren Ade,2016,https://www.imdb.com/title/tt4048272/,7.3,59.0,85.0,"['Psychological Drama', 'Quirky Comedy', 'Come..."
1,Requiem for a Dream,Darren Aronofsky,2000,https://www.imdb.com/title/tt0180093/,8.3,37.0,69.0,"['Psychological Drama', 'Tragedy', 'Drama']"
2,Carlos,Olivier Assayas,2010,https://www.imdb.com/title/tt1321865/,7.6,18.0,49.0,"['True Crime', 'Biography', 'Crime', 'Drama', ..."
3,The Gleaners and I,Agnès Varda,2000,https://www.imdb.com/title/tt0247380/,7.7,16.0,3.0,['Documentary']
4,Ten,Abbas Kiarostami,2002,https://www.imdb.com/title/tt0301978/,7.4,1.0,4.0,['Drama']
...,...,...,...,...,...,...,...,...
97,Boyhood,Richard Linklater,2014,https://www.imdb.com/title/tt1065073/,7.9,178.0,220.0,"['Coming-of-Age', 'Drama']"
98,Spirited Away,Hayao Miyazaki,2001,https://www.imdb.com/title/tt0245429/,8.6,58.0,31.0,"['Anime', 'Coming-of-Age', 'Fairy Tale', 'Hand..."
99,There Will Be Blood,Paul Thomas Anderson,2007,https://www.imdb.com/title/tt0469494/,8.2,114.0,138.0,"['Epic', 'Period Drama', 'Psychological Drama'..."
100,In the Mood for Love,Wong Kar-wai,2000,https://www.imdb.com/title/tt0118694/,8.1,45.0,50.0,"['Dark Romance', 'Tragic Romance', 'Drama', 'R..."


### filter filmd directed by men only

In [21]:
female_directors = [
    "Maren Ade", "Agnès Varda", "Claire Denis", 
    "Kathryn Bigelow", "Andrea Arnold", 
    "Lucrecia Martel", "Sarah Polley"
]

In [22]:
df_all = df_all[
    ~df_all["director"].isin(female_directors) & 
    ~df_all["director"].str.contains(",| and ", regex=True, na=False)
]

### convert ratings, nominations, wins to numerics

In [23]:
df_all['rating'] = pd.to_numeric(df_all['rating'], errors='coerce')

/var/folders/zz/2b8md6f924779tdhfsf8zvqm0000gn/T/ipykernel_29892/1801539845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['rating'] = pd.to_numeric(df_all['rating'], errors='coerce')


In [24]:
df_w['wins'] = pd.to_numeric(df_w['wins'], errors='coerce')

In [25]:
df_w['nominations'] = pd.to_numeric(df_w['nominations'], errors='coerce')

### the mean rating 

In [26]:
df_all['rating'].median()

7.8

### the number of awards won on average

In [27]:
df_all['wins'].median()

50.5

### the number of nominations on average

In [28]:
df_all['nominations'].median()

72.5

In [29]:
new_df_all = pd.DataFrame({
    'males': ['males'] * len(df_all),  # Repeat "females" for each row
    'rating': df_all['rating'],
    'wins': df_all['wins'],
    'director': df_all['director']
})

In [30]:
female_df = new_df.rename(columns={"females": "gender"})
male_df = new_df_all.rename(columns={"males": "gender"})

In [31]:
combined_df = pd.concat([female_df, male_df], ignore_index=True)

In [32]:
combined_df.to_csv('combined.csv', index=False)

In [214]:
df_all['Number_of_Genres'] = df_all['genres'].str.strip("[]").str.split(", ").apply(len)


AttributeError: Can only use .str accessor with string values!

In [79]:
bins = [0, 1999, 2005, 2010, 2015, 2020]  # 0 represents "before 2000"
labels = ['before 2000', '2000-2005', '2006-2010', '2011-2015', '2016-2020']

# Add 'Era' column
df_all['Era'] = pd.cut(
    df_all['year'],
    bins=bins,
    labels=labels,
    right=False
)

In [80]:
df_all['genres'] = df_all['genres'].str.strip("[]").str.replace("'", "").str.split(", ")


In [81]:
df_exploded_all = df_all.explode('genres')

In [82]:
genre_counts_all = df_exploded_all.groupby(['Era', 'genres']).size().reset_index(name='Count')

In [83]:
genre_counts_all = genre_counts_all[genre_counts_all['Count'] > 0]


In [84]:
genre_counts_all['genres'] = genre_counts_all['genres'].astype(str)  # Force string conversion


In [94]:
pd.set_option('display.max_rows', None)


In [95]:
genre_counts_all

,Era,genres,Count,Broad_Genre
98,2000-2005,Action,1,Action
99,2000-2005,Action Epic,2,Action
100,2000-2005,Adventure,5,Action
101,2000-2005,Adventure Epic,2,Action
103,2000-2005,Animal Adventure,1,Action
104,2000-2005,Animation,2,Other
105,2000-2005,Anime,1,Other
106,2000-2005,Artificial Intelligence,1,Other
107,2000-2005,Biography,1,Documentary
108,2000-2005,Buddy Comedy,1,Comedy


In [160]:
def map_to_parent(genre):
    if pd.isna(genre):
        return 'Other'
    genre_lower = str(genre).lower().strip()
    for parent, keywords in parent_genres.items():
        if any(keyword in genre_lower for keyword in keywords):  # ✅ Correct
            return parent
    return 'Other'

# Apply mapping
genre_counts_all['Broad_Genre'] = genre_counts_all['genres'].apply(map_to_parent)

# Aggregate
df_agg = genre_counts_all.groupby(['Era', 'Broad_Genre'], as_index=False)['Count'].sum()

In [161]:
df_agg

,Era,Broad_Genre,Count
0,before 2000,Action,0
1,before 2000,Animation,0
2,before 2000,Comedy,0
3,before 2000,Coming-of-Age,0
4,before 2000,Crime,0
5,before 2000,Documentary,0
6,before 2000,Family,0
7,before 2000,History,0
8,before 2000,Horror,0
9,before 2000,Mystery,0


In [167]:
# assign the gender to this dataframe

df_agg['Gender'] = 'Male' 

#### combined dataframes for both genders

In [208]:
df_datawrap = pd.concat([df_sorted, df_agg], ignore_index=True)

In [209]:
df_pivoted = df_datawrap.pivot_table(
    index=['Era', 'Broad_Genre'], 
    columns='Gender',              
    values='Count',               
    fill_value=0                  
).reset_index()                    

df_pivoted.columns = ['Era', 'Broad_Genre', 'Female', 'Male']

df_pivoted = df_pivoted.sort_values('Era')

In [210]:
df_pivoted

,Era,Broad_Genre,Female,Male
0,before 2000,Action,7.0,0.0
16,before 2000,War,3.0,0.0
15,before 2000,Thriller,11.0,0.0
14,before 2000,Sports,3.0,0.0
13,before 2000,Sci-Fi,9.0,0.0
11,before 2000,Psychological Drama,7.0,0.0
10,before 2000,Other,54.0,0.0
9,before 2000,Mystery,0.0,0.0
12,before 2000,Romance,15.0,0.0
7,before 2000,History,4.0,0.0


## Data analysis that made to my piece:
1. 

# THIS PART DID NOT CONTRIBUTE TO MY STORY - Analysis for Rotten Tomatoes List

In [110]:
df_rt = pd.read_csv('rotten.csv')

## 1. convert all statistics into numerics

In [111]:
df_rt['Year'] = pd.to_numeric(df_rt['Year'], errors='coerce').astype('Int64')

In [112]:
df_rt['Rating'] = pd.to_numeric(df_rt['Rating'], errors='coerce')

## 2. how many genred does each film is tagged?

In [113]:
df_rt['Number_of_Genres'] = df_rt['Genres'].str.strip("[]").str.split(", ").apply(len)


In [114]:
df_rt

,Title,Year,Director,Link,Rating,Wins,Nominations,Genres,Number_of_Genres
0,The Substance,2024,Coralie Fargeat,https://www.imdb.com/title/tt17526714/,7.3,143.0,280.0,"['Body Horror', 'Dark Comedy', 'Monster Horror...",8
1,My Old Ass,2024,Megan Park,https://www.imdb.com/title/tt18559464/,6.9,9.0,35.0,"['Coming-of-Age', 'Comedy', 'Drama', 'Romance']",4
2,Love Lies Bleeding,2024,Rose Glass,https://www.imdb.com/title/tt19637052/,6.6,6.0,53.0,"['Dark Romance', 'Drug Crime', 'Erotic Thrille...",10
3,The Fire Inside,2024,Rachel Morrison,https://www.imdb.com/title/tt6133444/,6.7,5.0,17.0,"['Boxing', 'Docudrama', 'Biography', 'Drama', ...",5
4,The Devil's Bath,2024,Veronika Franz,https://www.imdb.com/title/tt29141112/,6.6,15.0,10.0,"['Folk Horror', 'Period Drama', 'Drama', 'Hist...",6
...,...,...,...,...,...,...,...,...,...
283,Monsoon Wedding,2001,Mira Nair,https://www.imdb.com/title/tt0265343/,7.3,7.0,12.0,"['Feel-Good Romance', 'Romantic Comedy', 'Come...",5
284,Promises,2001,Justine Shapiro,https://www.imdb.com/title/tt0282864/,8.3,15.0,5.0,['Documentary'],1
285,The Taste of Others,2000,Agnès Jaoui,https://www.imdb.com/title/tt0216787/,7.2,16.0,12.0,"['Comedy', 'Drama', 'Romance']",3
286,The Gleaners and I,2000,Agnès Varda,https://www.imdb.com/title/tt0247380/,7.7,16.0,3.0,['Documentary'],1


## 3. divide the data by eras

In [115]:
bins = [1999, 2010, 2020, 2025]
labels = ['1999-2010', '2010-2020', '2020-2024']

# Add 'Era' column
df_rt['Era'] = pd.cut(
    df_rt['Year'],
    bins=bins,
    labels=labels,
    right=False
)

In [116]:
df_rt

,Title,Year,Director,Link,Rating,Wins,Nominations,Genres,Number_of_Genres,Era
0,The Substance,2024,Coralie Fargeat,https://www.imdb.com/title/tt17526714/,7.3,143.0,280.0,"['Body Horror', 'Dark Comedy', 'Monster Horror...",8,2020-2024
1,My Old Ass,2024,Megan Park,https://www.imdb.com/title/tt18559464/,6.9,9.0,35.0,"['Coming-of-Age', 'Comedy', 'Drama', 'Romance']",4,2020-2024
2,Love Lies Bleeding,2024,Rose Glass,https://www.imdb.com/title/tt19637052/,6.6,6.0,53.0,"['Dark Romance', 'Drug Crime', 'Erotic Thrille...",10,2020-2024
3,The Fire Inside,2024,Rachel Morrison,https://www.imdb.com/title/tt6133444/,6.7,5.0,17.0,"['Boxing', 'Docudrama', 'Biography', 'Drama', ...",5,2020-2024
4,The Devil's Bath,2024,Veronika Franz,https://www.imdb.com/title/tt29141112/,6.6,15.0,10.0,"['Folk Horror', 'Period Drama', 'Drama', 'Hist...",6,2020-2024
...,...,...,...,...,...,...,...,...,...,...
283,Monsoon Wedding,2001,Mira Nair,https://www.imdb.com/title/tt0265343/,7.3,7.0,12.0,"['Feel-Good Romance', 'Romantic Comedy', 'Come...",5,1999-2010
284,Promises,2001,Justine Shapiro,https://www.imdb.com/title/tt0282864/,8.3,15.0,5.0,['Documentary'],1,1999-2010
285,The Taste of Others,2000,Agnès Jaoui,https://www.imdb.com/title/tt0216787/,7.2,16.0,12.0,"['Comedy', 'Drama', 'Romance']",3,1999-2010
286,The Gleaners and I,2000,Agnès Varda,https://www.imdb.com/title/tt0247380/,7.7,16.0,3.0,['Documentary'],1,1999-2010
